In [1]:
import pandas as pd
import numpy as np
import os
import math
import statistics as stats
import gurobipy as gb
from gurobipy import *

### Read in Data

In [2]:
# set to your working directory
dir = "/Users/adityodasgupta/Documents/McGill/MGSC 662/Group"
os.chdir(dir)
dft = pd.read_excel("Model_Data.xlsx", sheet_name="Transitional")
dfe = pd.read_excel("Model_Data.xlsx", sheet_name="Emergency")
dfh = pd.read_excel("Model_Data.xlsx", sheet_name="HotSpots")
dfn = pd.read_excel("Model_Data.xlsx", sheet_name="NewLocations")

### Define Haversine Distance Calculation

In [3]:
def haversine_distance(coord1: list, coord2: list):
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c 
    km = meters / 1000.0

    meters = round(meters, 3)
    km = round(km, 3)
    miles = km * 0.621371
    return km

### Data Preparation

In [4]:
# 50% of homeless people assigned to metros
# 50% of homeless people assigned to homeless camps
pct = 0.5

total_homeless = 3573
total_metro = math.floor(total_homeless * pct)
total_camp = total_homeless - total_metro

### Transitional

In [5]:
# Calculate Total Number of beds
dft_piv = pd.pivot_table(dft, values="Beds", index="Index", aggfunc=np.sum)
dft_piv.columns = ["Total Beds"]
dft = pd.merge(dft, dft_piv, how = "left", left_on="Index", right_on="Index")

# Replace missing values with median value of costs
dft['Annual Cost'] = dft['Annual Cost'].replace({np.nan:dft['Annual Cost'].median()})

# calculate adjusted costs based on number of beds
dft['Adjusted Costs'] = (dft['Beds'] / dft['Total Beds']) * dft['Annual Cost']

# calculate the variable cost
# not needed
var_cost = 25860
dft['Variable Cost'] = dft['Beds'] * var_cost

dft = dft.drop(columns="Index")

### Emergency

In [6]:
# Calculate Total Number of Beds
dfe_piv = pd.pivot_table(dfe, values="Beds", index="Index", aggfunc=np.sum)
dfe_piv.columns = ["Total Beds"]
dfe = pd.merge(dfe, dfe_piv, how = "left", left_on="Index", right_on="Index")

# replace missing values with median value of costs
dfe['Annual Cost'] = dfe['Annual Cost'].replace({np.nan:dfe['Annual Cost'].median()})

# calculated adjusted costs based on number of beds
dfe['Adjusted Costs'] = (dfe['Beds'] / dfe['Total Beds']) * dfe['Annual Cost']

# calculate the variable cost
# not needed
var_cost = 25860
dfe['Variable Cost'] = dfe['Beds'] * var_cost

dfe = dfe.drop(columns="Index")

### Combine Transitional and Emergency

In [7]:
dfc = pd.concat([dft, dfe])

### Hotspots

In [8]:
dfh_camp = dfh[pd.isna(dfh['Density'])]
dfh_camp = dfh_camp[['Station', 'Density', 'Latitude', 'Longitude']].reset_index()
dfh_metro = dfh[pd.notna(dfh['Density'])]
dfh_metro = dfh_metro[['Station', 'Density', 'Latitude', 'Longitude']].reset_index()

In [9]:
# distribute homeless in camps equally except West Island
num_camps = dfh_camp.shape[0]
dfh_camp['Homeless'] = (total_camp - 25) // (num_camps - 1)
dfh_camp.loc[dfh_camp['Station'] == "West Island",'Homeless'] = 25 # Set West Island to 25

dfh_camp['Male'] = dfh_camp['Homeless'] * 0.7
dfh_camp['Male'] = dfh_camp['Male'].round() + 1

dfh_camp['Female'] = dfh_camp['Homeless'] * 0.3
dfh_camp['Female'] = dfh_camp['Female'].round()

dfh_camp.loc[dfh_camp['Station'] == "West Island",'Male'] = 18
dfh_camp.loc[dfh_camp['Station'] == "West Island",'Female'] = 7

dfh_camp = dfh_camp.drop(columns=['Density'])

In [10]:
# find the minimum density value
# put density of metro station divided by minimum density
low = dfh_metro['Density'].min()
dfh_metro['Scale'] = dfh_metro['Density'] / low
total = dfh_metro['Scale'].sum()

# calculate homeless distribution
dfh_metro['Homeless'] = dfh_metro['Scale'] / total * total_metro
dfh_metro['Homeless'] = dfh_metro['Homeless'].round()

# determine portion male and portion female
dfh_metro['Male'] = dfh_metro['Homeless'] * 0.7
dfh_metro['Male'] = dfh_metro['Male'].round()

dfh_metro['Female'] = dfh_metro['Homeless'] * 0.3
dfh_metro['Female'] = dfh_metro['Female'].round()

dfh_metro = dfh_metro.drop(columns=['Density', 'Scale'])

In [11]:
# Concatenate metro and camps
dfd = pd.concat([dfh_metro, dfh_camp]).reset_index()

### Potential

In [12]:
dfn.head()

,Location,Website,Latitude,Longitude,Capacity
0,Hotel des Arts,https://globalnews.ca/news/8530589/shelter-ind...,45.512588,-73.567747,50
1,Guy-Favreau complex downtown where fifty beds ...,https://globalnews.ca/news/8530589/shelter-ind...,45.506624,-73.562953,50
2,in Montreal. An additional 300 places will be ...,https://montreal.ctvnews.ca/montreal-mayor-say...,45.562380,-73.602061,300
3,"Last year, she said the city opened more tempo...",https://globalnews.ca/news/8495022/montreal-ho...,45.509150,-73.551544,108
4,Old royal victoria's hopital,https://montreal.ctvnews.ca/old-royal-victoria...,45.473405,-73.601579,80


### Export to Excel

In [13]:
with pd.ExcelWriter('Final_Model_Data.xlsx') as writer:
    dft.to_excel(writer, sheet_name="Transitional", index=False)
    dfe.to_excel(writer, sheet_name="Emergency", index=False)
    dfc.to_excel(writer, sheet_name="CombinedHousing", index=False)
    dfh_camp.to_excel(writer, sheet_name="HotspotCamps", index=False)
    dfh_metro.to_excel(writer, sheet_name="HotspotsMetro", index=False)
    dfd.to_excel(writer, sheet_name="HotspotsAll", index=False)
    dfn.to_excel(writer, sheet_name="NewLocations", index=False)

### Model 1: Simulation

#### Model 1 Data Prep

In [14]:
hotspot = dfd.copy()
shelter = dfc.copy().reset_index()

In [15]:
shelter.sort_values(by='Code',inplace=True) # sort by Code to segment All, Female and Male shelters
shelter.reset_index(inplace=True)
shelter.drop(columns='index',inplace=True)

In [16]:
Shelter = shelter.loc[:,'Beds'].tolist()
Females = hotspot.loc[:,'Female'].tolist()
Males = hotspot.loc[:,'Male'].tolist()
Gender = ['Female', 'Male']

In [17]:
H = len(hotspot) # number of hotspots
S = len(shelter) # number of shelters
G = len(Gender) # number of genders

In [18]:
Distance=[]
for i in range(H):
    l=[]
    hotspot_coord=[hotspot.loc[i,'Latitude'],hotspot.loc[i,'Longitude']]
    for j in range(S):
        shelter_coord=[shelter.loc[j,'Latitude'],shelter.loc[j,'Longitude']]
        
        dis=haversine_distance(hotspot_coord,shelter_coord)
        l.append(dis)
    Distance.append(l)

In [19]:
A = shelter[shelter['Code']==0].shape[0] # Number of shelters open to all
F = shelter[shelter['Code']==1].shape[0] # Number of shelters open to only females
M = shelter[shelter['Code']==2].shape[0] # Number of shelters open to only males

#### Add Decision Variables and Set Objective Function

In [20]:
prob = gb.Model("Shelters")

X = prob.addVars(H, S, G, lb=0, vtype=GRB.INTEGER, name=['X_Hotspot_'+str(i)+'_toShelter_'+str(j)+'_'+k for i in range(1,H+1) for j in range(1,S+1) for k in Gender])

prob.setObjective(gb.quicksum(X[i,j,k] for i in range(H) for j in range(S) for k in range(G)),GRB.MAXIMIZE)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-15


#### Implement Constraints

In [21]:
# Homeless People Constraints
# Female shelters can only have Females
for j in range(A,A+F):
    prob.addConstr(sum(X[i,j,0] for i in range(H))<=Shelter[j])

# Stop Males from going to Female Shelters
for j in range(A,A+F):
    prob.addConstr(sum(X[i,j,1] for i in range(H))==0)  

# Male shelters can only have Males
for j in range(A+F,A+F+M):
    prob.addConstr(sum(X[i,j,1] for i in range(H))<=Shelter[j])

# Stop Females from going to Male Shelters
for j in range(A+F,A+F+M):
    prob.addConstr(sum(X[i,j,0] for i in range(H))==0)

#Males in Hotspots
for i in range(H):
    prob.addConstr(sum(X[i,j,1] for j in range(S))<=Males[i])

#FeMales in Hotspots
for i in range(H):
    prob.addConstr(sum(X[i,j,0] for j in range(S))<=Females[i])

# All
for j in range(0,A):
    prob.addConstr(sum(X[i,j,k] for i in range(H) for k in range(G))<=Shelter[j])

In [22]:
# Distance constraints between hotspots and shelters
for i in range(H):
    for j in range(S):
        for k in range(G):
            prob.addConstr(X[i,j,k]*(4-Distance[i][j])>=0) #km

In [23]:
prob.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12096 rows, 11826 columns and 35478 nonzeros
Model fingerprint: 0xb926459d
Variable types: 0 continuous, 11826 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-03, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 3e+02]
Found heuristic solution: objective 3016.0000000
Presolve removed 11921 rows and 8514 columns
Presolve time: 0.04s
Presolved: 175 rows, 3312 columns, 6623 nonzeros
Variable types: 0 continuous, 3312 integer (0 binary)

Root relaxation: objective 3.142000e+03, 388 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3142.0000000 3142.00000  0.00%     -    0s

Explored 1 n

In [24]:
print(f'The total number of people in shelters : {prob.ObjVal}')
print(f'The optimal allocation is ')
for i in prob.getVars():
        print(f"{i.VarName}: {i.X}")

The total number of people in shelters : 3142.0
The optimal allocation is 
X_Hotspot_1_toShelter_1_Female: 0.0
X_Hotspot_1_toShelter_1_Male: 0.0
X_Hotspot_1_toShelter_2_Female: 0.0
X_Hotspot_1_toShelter_2_Male: 0.0
X_Hotspot_1_toShelter_3_Female: 0.0
X_Hotspot_1_toShelter_3_Male: 0.0
X_Hotspot_1_toShelter_4_Female: 0.0
X_Hotspot_1_toShelter_4_Male: 0.0
X_Hotspot_1_toShelter_5_Female: 0.0
X_Hotspot_1_toShelter_5_Male: 0.0
X_Hotspot_1_toShelter_6_Female: -0.0
X_Hotspot_1_toShelter_6_Male: -0.0
X_Hotspot_1_toShelter_7_Female: 0.0
X_Hotspot_1_toShelter_7_Male: 0.0
X_Hotspot_1_toShelter_8_Female: 0.0
X_Hotspot_1_toShelter_8_Male: 0.0
X_Hotspot_1_toShelter_9_Female: 0.0
X_Hotspot_1_toShelter_9_Male: 0.0
X_Hotspot_1_toShelter_10_Female: -0.0
X_Hotspot_1_toShelter_10_Male: -0.0
X_Hotspot_1_toShelter_11_Female: -0.0
X_Hotspot_1_toShelter_11_Male: -0.0
X_Hotspot_1_toShelter_12_Female: 0.0
X_Hotspot_1_toShelter_12_Male: 0.0
X_Hotspot_1_toShelter_13_Female: 0.0
X_Hotspot_1_toShelter_13_Male: 0.0
X

In [33]:
# calculate the number remaining at each hotspot from model 1 as input to model 2
rem_male=[]
rem_female=[]
for i in range(H):
    f=0
    m=0
    for j in range(S):
        f = f + X[i,j,0].x 
        m = m + X[i,j,1].x
    rem_male.append(m)
    rem_female.append(f)

hotspot['Remaining']=hotspot['Homeless']-np.array(rem_male)-np.array(rem_female)
hotspot['Female_Rem']=hotspot['Female']-np.array(rem_female)
hotspot['Male_Rem']=hotspot['Male']-np.array(rem_male)

In [30]:
# calculate the number of beds unoccupied at each shelter
unocc=[]
for i in range(S):
    s=0
    for j in range(H):
        s = s + X[j,i,0].x + X[j,i,1].x
    unocc.append(s)

shelter['Remaining']=shelter['Total Beds']-np.array(unocc)

In [32]:
shelter[shelter['Remaining']>0]

,level_0,Code,Name,ADDRESS,Beds,Latitude,Longitude,Annual Cost,Total Beds,Adjusted Costs,Variable Cost,Remaining
0,0,0,AMCAL FAMILY SERVICES - RESIDENTIAL PROGRAM,"7 Sainte-Anne Street, Pointe-Claire, Montréal,...",63,45.428834,-73.824636,1296983.0,63,1.296983e+06,1629180,63.0
2,38,0,PAS DE LA RUE (LE),"1575 René-Lévesque Boulevard East, Ville-Marie...",45,45.520640,-73.551029,802244.0,79,4.569744e+05,1163700,34.0
3,39,0,PAS DE LA RUE (LE) - POINT DE SERVICE DE MERCI...,"9605 Hochelaga Street East, Mercier—Hochelaga-...",34,45.611177,-73.515385,802244.0,79,3.452696e+05,879240,45.0
6,43,0,RESSOURCES JEUNESSE DE SAINT-LAURENT - APPARTE...,"1550 Élizabeth Street, Suite 100, Saint-Lauren...",33,45.523578,-73.683726,794150.0,45,5.823767e+05,853380,12.0
7,44,0,RESSOURCES JEUNESSE DE SAINT-LAURENT - MAISON ...,"1410 O'Brien Street, Saint-Laurent, Montréal, ...",12,45.519173,-73.686288,794150.0,45,2.117733e+05,310320,33.0
8,45,0,RICOCHET (HÉBERGEMENT/HOMES) - SOCIAL INTEGRAT...,"5100 du Château-Pierrefonds Avenue, Pierrefond...",20,45.464937,-73.897389,255797.0,20,2.557970e+05,517200,7.0
12,51,0,VILAVI - PHASE I - MAISON DE CHAMBRES ST-ANDRÉ,"1271-1275 Saint-André Street, Ville-Marie, Mon...",20,45.516276,-73.557299,2010460.0,91,4.418593e+05,517200,71.0
13,53,0,VILAVI - PHASE III - STUDIOS VIGER,"901 Viger Avenue East, Mercier—Hochelaga-Maiso...",25,45.515104,-73.552692,2010460.0,91,5.523242e+05,646500,66.0
14,54,0,VILAVI - PHASE IV - MAISON DE CHAMBRES LESPÉRANCE,"2190-2200 Lespérance Street, Ville-Marie, Mont...",15,45.538504,-73.552017,2010460.0,91,3.313945e+05,387900,76.0
15,55,0,VILAVI - PHASE V - MAISON DE CHAMBRES MONTCALM,"1269 Montcalm Street, Ville-Marie, Montréal, Q...",10,45.518103,-73.555181,2010460.0,91,2.209297e+05,258600,81.0


### Model 2: New Shelter Allocation

#### Data Prep

In [68]:
# use hotspot data from previous Model
potential = dfn.copy().reset_index()
shelter = dfc.copy().reset_index()
shelter.sort_values(by='Code',inplace=True) # sort by Code to segment All, Female and Male shelters
shelter.reset_index(inplace=True)
shelter.drop(columns='index',inplace=True)

In [69]:
leftover=hotspot[hotspot['Remaining']>0].copy()
leftover['Annual_Cost_Homeless']=25860
leftover.drop(columns=['level_0','index'],inplace=True)
leftover.reset_index(inplace=True)
leftover.drop(columns='index',inplace=True)

In [70]:
P = len(potential)
S = len(shelter)
L = len(leftover)

In [71]:
# Calculate the fixed cost of new shelters by taking average of 3 closest shelters in original data set
dummy=pd.DataFrame({'Location':[],'Shelter':[],'Distance':[],'Adjusted Cost':[]})

for i in range(P):
    potential_coord=[potential.loc[i,'Latitude'],potential.loc[i,'Longitude']]
    for j in range(S):
        shelter_coord=[shelter.loc[j,'Latitude'],shelter.loc[j,'Longitude']]
        
        dis=haversine_distance(potential_coord,shelter_coord)
        dummy=pd.concat([dummy,pd.DataFrame({'Location':potential.loc[i,'Location'],'Shelter':shelter.loc[j,'Name'],
                      'Distance':dis,'Adjusted Cost':shelter.loc[j,'Adjusted Costs']},index=[0])])

dummy.reset_index(inplace=True)
dummy.drop(columns='index',inplace=True)


In [72]:
dummy['row_num']=dummy.groupby(by='Location')['Distance'].rank(method='first')
dummy=dummy[dummy['row_num'].isin([1,2,3])]
dum=pd.DataFrame(dummy.groupby(by='Location')['Adjusted Cost'].mean())
potential=potential.join(dum,how='inner',on='Location')

In [73]:
Distance=[]
for i in range(L):
    l=[]
    leftover_coord=[leftover.loc[i,'Latitude'],leftover.loc[i,'Longitude']]
    for j in range(P):
        potential_coord=[potential.loc[j,'Latitude'],potential.loc[j,'Longitude']]
        
        dis=haversine_distance(leftover_coord,potential_coord)
        l.append(dis)
    Distance.append(l)

In [74]:
Shelters = potential.loc[:,'Capacity'].tolist()
Homeless = leftover.loc[:,'Remaining'].tolist()
Fixed_Cost = potential.loc[:,'Adjusted Cost'].tolist()
Cost_Per_Homeless = leftover.loc[:,'Annual_Cost_Homeless'].tolist()

#### Create Decision Variables and Set Objective Function

In [75]:
prob1 = gb.Model("Shelter model 2")

X = prob1.addVars(L, P, lb=0, vtype=GRB.INTEGER, name=['X_LOC_'+str(i+1)+'_toShelter_'+str(j+1) for i in range(L) for j in range(P)])
D = prob1.addVars(P, vtype=GRB.BINARY)

prob1.setObjective(sum(Cost_Per_Homeless[i]*X[i,j] for i in range(L) for j in range(P))+sum(Fixed_Cost[i]*D[i] for i in range(P)),GRB.MINIMIZE)

# Add Constraints to Model

In [76]:
prob1.addConstr(sum(D[i] for i in range(P))<=7)  #only one of the three locations will be picked

for i in range(L):
    for j in range(P):
        prob1.addConstr(X[i,j]*(4-Distance[i][j])>=0) #distance constraint
        
#sum of homeless allocated has to be less than or equal to the capacity
for j in range(P):
    prob1.addConstr(sum(X[i,j] for i in range(L))<=Shelters[j])

# sum of homeless allocated has to be less than or equal to the number of homeless remaining at hotspots
for i in range(L):
    prob1.addConstr(sum(X[i,j] for j in range(P))<=Homeless[i])
    
# all homeless have to be allocated to a shelter
prob1.addConstr(sum(X[i,j] for i in range(L) for j in range(P))>= 0.85*sum(Homeless[i] for i in range(L)))

# a shelter can only host homeless if it is chosen
M=1000000
for i in range(L):
     for j in range(P):
        prob1.addConstr(X[i,j]<=D[j]*M)

prob1.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 234 rows, 112 columns and 637 nonzeros
Model fingerprint: 0x3db78231
Variable types: 0 continuous, 112 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+06]
  Objective range  [3e+04, 9e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective 2.178011e+07
Presolve removed 217 rows and 84 columns
Presolve time: 0.00s
Presolved: 17 rows, 28 columns, 81 nonzeros
Variable types: 0 continuous, 28 integer (4 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 2.17801e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.178010779167e+07, best bound 2.178010779167e+07, gap 0.0000%


In [77]:
for v in prob1.getVars():
    if v.x>0:
        print(v.varName, "=", v.x)

X_LOC_2_toShelter_5 = 62.0
X_LOC_3_toShelter_3 = 21.0
X_LOC_4_toShelter_3 = 11.0
X_LOC_5_toShelter_3 = 1.0
X_LOC_6_toShelter_3 = 41.0
X_LOC_8_toShelter_3 = 18.0
X_LOC_9_toShelter_3 = 19.0
X_LOC_10_toShelter_3 = 22.0
X_LOC_11_toShelter_3 = 16.0
X_LOC_13_toShelter_3 = 23.0
X_LOC_14_toShelter_3 = 33.0
X_LOC_15_toShelter_3 = 95.0
C107 = 1.0
C109 = 1.0


In [78]:
t=[]

for i in range(len(leftover)):
    s=0
    for j in range(len(potential)):
        s=s+X[i,j].x
    t.append(s)
print(f'The total people newly assigned to shelters are {sum(t)}')

The total people newly assigned to shelters are 362.0


In [79]:
leftover['Remaining 1']=leftover['Remaining']-np.array(t)
leftover[leftover['Remaining 1'] > 0]

,Station,Latitude,Longitude,Homeless,Male,Female,Remaining,Annual_Cost_Homeless,Remaining 1
0,Plamondon,45.50484,-73.6379,27.0,19.0,8.0,19.0,25860,19.0
1,Cote-Sainte-Catherine,45.50436,-73.6349,67.0,47.0,20.0,67.0,25860,5.0
6,Cremazie,45.55631,-73.6404,29.0,20.0,9.0,20.0,25860,20.0
7,Cote-des-Neiges,45.50821,-73.6239,29.0,20.0,9.0,20.0,25860,2.0
11,Acadie,45.53130,-73.6246,17.0,12.0,5.0,17.0,25860,17.0


In [80]:
print('The total people still unallocated : ',sum(leftover['Remaining 1']))

The total people still unallocated :  63.0


In [81]:
print('The total optimal cost : ',prob1.objVal)

The total optimal cost :  21780107.791666664
